In [2]:
import pandas as pd
from transformers import *
import swifter

Neither PyTorch nor TensorFlow >= 2.0 have been found.Models won't be available and only tokenizers, configurationand file/data utilities can be used.


In [2]:
def calculate_text(row):
    tweet_tokens = tokenizer.decode(list(map(int, row.split('\t'))))
    return tweet_tokens

In [3]:
def tokenize_text(row, index):
    if index % 100000 == 0:
        print(index)
    row1 = row.replace('[CLS]','')
    row2 = row1.replace('[SEP]', '')
    result = re.sub(r"http\S+", "", row2)
    tokenizer = RegexpTokenizer(r'\w+')
    token_list = tokenizer.tokenize(result)
    feature =  ' '.join([w.lower() for w in token_list])
    return feature

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [5]:
all_features = ["text_ tokens"]

test = pd.read_csv("s3://recsys-challenge-2020/competition_test.tsv", encoding="utf-8",
                    names = all_features, usecols= [0], sep="\x01")
test_text = pd.DataFrame()
test_text['user_text'] = test['text_ tokens'].swifter.apply(lambda x: calculate_text(x))

NameError: name 're' is not defined

In [9]:
import re
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer

In [10]:
cleaned_text_test = pd.DataFrame()
cleaned_text_test['tweet_text'] = test_text.swifter.apply(lambda x: tokenize_text(x.user_text, x.name), axis = 1)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000

In [11]:
all_features = ["language"]
test_language = pd.read_csv("s3://recsys-challenge-2020/competition_test.tsv", encoding="utf-8",
                    names = all_features, usecols= [7], sep="\x01"
                   )
print('test language loaded')
language_code_name_pairs = pd.read_csv('s3://recsys-challenge-2020/language_code_name_pairs.csv')
test_clean_input = pd.concat([test_language[['language']], cleaned_text_test], axis=1)

test language loaded


KeyError: 'Language'

In [12]:
test_input = pd.merge(test_clean_input, language_code_name_pairs, how = 'inner',\
left_on = 'language', right_on = 'code')[['tweet_text', 'name']]
test_input.to_csv('s3://recsys-challenge-2020/competetion_test_input.csv', index = False)

In [14]:
test_input =  test_input.sort_values(by='name')

In [23]:
test_input.to_csv('/home/ubuntu/recsys_challenge_2020/competetion_submission_phase/data/test_input', index = False)

In [3]:
test_input = pd.read_csv('/home/ubuntu/recsys_challenge_2020/competetion_submission_phase/data/test_input')

In [4]:
languages = list(test_input.name.unique())

In [5]:
import fasttext
import fasttext.util
def get_fasttext_sentence_embedding(row, ft):
    if pd.isna(row):
        return np.zeros(20)
    return ft.get_sentence_vector(row)

In [ ]:
all_lang_output = pd.DataFrame()
for language in languages:
    print('starting langage: ' + language)
    lang_output = pd.DataFrame()
    lang_input = test_input.loc[test_input.name == language]
    print('number of lines to compute: ' + str(len(lang_input)))
    fasttext.util.download_model(language, if_exists='ignore')  # English
    ft = fasttext.load_model('cc.'+language+'.300.bin')
    fasttext.util.reduce_model(ft, 20)
    lang_output['sentence_embedding'] = lang_input.swifter.apply(lambda x: get_fasttext_sentence_embedding(x.tweet_text, ft), axis = 1)
    all_lang_output = pd.concat([all_lang_output, lang_output])
    print('finished language: ' + language)

starting langage: af
number of lines to compute: 25160


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

